In [ ]:
import pandas as pd
import sqlalchemy as sql
import seaborn as sns
import matplotlib.pyplot as plt
import umap
import sklearn as sk
import kagglehub
import os
import plotly.express as px
import plotly.graph_objects as go
import sqlalchemy as sql

### Licence

I used Two datasets about origin citizenships of internashional studetns in canada and usa.

Both The us and canadian international students datasets license is CC0: Public Domain

- No Copyright: anyone can use them for any purpose.

- Free Use: Users can copy, modify, distribute, and perform the work, even for commercial purposes, without any restrictions.

- Global Applicability: CC0 is intended to apply globally, although the specifics may vary based on local copyright laws.

- No Attribution Required: it is not legally required when using CC0 works.


 


## Data pipeline

The whole pipeline is in python and is structured as follows.

Getting the data -> Processing the data to two dataframes (meta and usage data) -> Transform data -> Store data




### Getting the data

The pipeline starts with a GET request to the provided url. I used pre-writen kaggle function to download CSV files.


In [3]:

data_path= '/home/cip/ce/ix05ogym/Majid/MADE/data/'
urls = [
        "syedabdulshameer/international-students-in-canada",
        "webdevbadger/international-student-demographics",
        'justin2028/unemployment-in-america-per-us-state'
        ]

for url in urls:
    path = kagglehub.dataset_download(url)
    os.system(f'cp -r {path}/* {data_path}')
    print("Path to dataset files:", path)
    os.listdir(path)
    
    


Path to dataset files: /home/cip/ce/ix05ogym/.cache/kagglehub/datasets/syedabdulshameer/international-students-in-canada/versions/2
Path to dataset files: /home/cip/ce/ix05ogym/.cache/kagglehub/datasets/webdevbadger/international-student-demographics/versions/2
Path to dataset files: /home/cip/ce/ix05ogym/.cache/kagglehub/datasets/justin2028/unemployment-in-america-per-us-state/versions/3


### Load Data

In [4]:
csv_names= os.listdir(data_path)
df ={}
for name in csv_names:
    name = name.split('.')
    extention = name[1]
    name = name[0]
    if extention != 'csv':
        continue
    
    print(name)
    df[name] = pd.read_csv(data_path+name+'.csv')
    #display(df[name])
    

Internation_students_canada = df['Internation_students_Canada']
Internation_students_us =df['origin']

origin
field_of_study
academic_detail
Internation_students_Canada
Unemployment in America Per US State
Internation_students_Province_Canada
source_of_fund
status
International_Students_Study_level
academic


### visualise data

In [5]:
Internation_students_canada.head()

,Country of Citizenship,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,95,115,95,80,95,90,80,170,140
1,Albania,115,165,185,245,375,250,305,345,545
2,Algeria,1060,845,1020,1490,2690,2170,3165,5360,7180
3,Andorra,0,0,0,0,0,0,10,5,0
4,Angola,65,80,40,25,120,30,50,75,65


In [6]:
Internation_students_us.head()

,year,origin_region,origin,academic_type,students
0,2000/01,"Africa, Subsaharan","Africa, Subsaharan, Unspecified",Graduate,2
1,2000/01,"Africa, Subsaharan","Africa, Subsaharan, Unspecified",Other,0
2,2000/01,"Africa, Subsaharan","Africa, Subsaharan, Unspecified",Undergraduate,6
3,2000/01,Asia,"Asia, Unspecified",Graduate,0
4,2000/01,Asia,"Asia, Unspecified",Other,6



### Processing and Transform the data

- it is important to check is there is any NaN for null field in the datasets. 
- unify the column names of two datasets
- Then we unify the datatypes of two datasets to make them prepare for merging. 
- To merge two dataset we have to calculate all the international students no matter what their study level are. for this we use groupby and sum of the students in all the studying level.
- Then I merged two datasets based on origin citizenship of international students.


In [7]:
print(Internation_students_canada.isnull().any().any(),
Internation_students_canada.isna().any().any())


False False


In [8]:
print(Internation_students_us.isnull().any().any(),
Internation_students_us.isna().any().any())

False False


In [9]:
Internation_students_us =Internation_students_us.groupby(['origin','year']).agg({'students': 'sum'}).reset_index()#.plot(kind='bar', figsize=(12, 8), width=0.8, stacked=True,legend=False,)
Internation_students_us['year'] = Internation_students_us['year'].str.split('/').str[0]
Internation_students_us.head()

,origin,year,students
0,Afghanistan,2000,75
1,Afghanistan,2001,92
2,Afghanistan,2002,102
3,Afghanistan,2003,109
4,Afghanistan,2004,155


In [10]:
Internation_students_us = Internation_students_us.pivot(index='origin', columns='year', values='students').fillna(0)
Internation_students_us.head()

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
origin,,,,,,,,,,,,,,,,,,,,,
Afghanistan,75.0,92.0,102.0,109.0,155.0,175.0,268.0,338.0,407.0,426.0,...,450.0,471.0,498.0,506.0,476.0,418.0,422.0,354.0,420.0,682.0
"Africa, Subsaharan, Unspecified",8.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,28.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,1117.0,1129.0,1106.0,916.0,866.0,904.0,841.0,689.0,757.0,790.0,...,636.0,670.0,705.0,781.0,884.0,993.0,1095.0,1053.0,1026.0,1087.0
Algeria,220.0,196.0,177.0,148.0,143.0,132.0,145.0,179.0,172.0,178.0,...,173.0,175.0,158.0,192.0,212.0,238.0,239.0,219.0,268.0,302.0
Andorra,5.0,8.0,11.0,9.0,11.0,6.0,3.0,9.0,12.0,11.0,...,9.0,9.0,11.0,16.0,14.0,17.0,11.0,16.0,18.0,20.0


In [11]:
Internation_students_canada.rename(columns={'Country of Citizenship':'origin'}, inplace=True)
#Internation_students_canada

In [18]:
merged_table = pd.merge(Internation_students_us, Internation_students_canada, on='origin', how='inner', suffixes=('', '_Canada'))
merged_table.iloc[:, 1:] = merged_table.iloc[:, 1:].astype('float')
merged_table = merged_table[['origin',
                             '2015','2016', '2017', '2018', '2019', '2020', '2021', '2022',
                             '2015_Canada','2016_Canada', '2017_Canada', '2018_Canada', '2019_Canada','2020_Canada', '2021_Canada', '2022_Canada']]
merged_table.head()


/tmp/ipykernel_201416/3652320.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        95.0
1       115.0
2      1060.0
3         0.0
4        65.0
        ...  
149    2980.0
150       0.0
151     175.0
152     120.0
153     335.0
Name: 2015_Canada, Length: 154, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_table.iloc[:, 1:] = merged_table.iloc[:, 1:].astype('float')
/tmp/ipykernel_201416/3652320.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       115.0
1       165.0
2       845.0
3         0.0
4        80.0
        ...  
149    5495.0
150       0.0
151     210.0
152     125.0
153     500.0
Name: 2016_Canada, Length: 154, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_table.iloc[:, 1:] = merged_table.il

,origin,2015,2016,2017,2018,2019,2020,2021,2022,2015_Canada,2016_Canada,2017_Canada,2018_Canada,2019_Canada,2020_Canada,2021_Canada,2022_Canada
0,Afghanistan,498.0,506.0,476.0,418.0,422.0,354.0,420.0,682.0,95.0,115.0,95.0,80.0,95.0,90.0,80.0,170.0
1,Albania,705.0,781.0,884.0,993.0,1095.0,1053.0,1026.0,1087.0,115.0,165.0,185.0,245.0,375.0,250.0,305.0,345.0
2,Algeria,158.0,192.0,212.0,238.0,239.0,219.0,268.0,302.0,1060.0,845.0,1020.0,1490.0,2690.0,2170.0,3165.0,5360.0
3,Andorra,11.0,16.0,14.0,17.0,11.0,16.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,5.0
4,Angola,1296.0,1257.0,1183.0,1014.0,814.0,673.0,541.0,553.0,65.0,80.0,40.0,25.0,120.0,30.0,50.0,75.0


### Extract data into sqllite

In [15]:
engine = sql.create_engine('sqlite:///international_students_north_america.db')
merged_table.to_sql('international_students_north_america', engine, if_exists='replace', index=False)

154

### Analysis the data

In [14]:
us = merged_table[['2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022']]

canada_column = ['2015_Canada',
       '2016_Canada', '2017_Canada', '2018_Canada', '2019_Canada',
       '2020_Canada', '2021_Canada', '2022_Canada']
canada = merged_table[canada_column]

canada.columns = [col.replace('_Canada', '') for col in canada.columns]


us = us.astype('float')
canada = canada.astype('float')

In [ ]:
#pd.set_option('display.max_rows', 200)

corr = us.corrwith(canada,1)
result = pd.concat([merged_table['origin'], corr], axis=1,).rename(columns={0:'corr'})
merged_table = merged_table.merge(result)
columns_to_select = ['origin','corr'] +  list(us.columns) + canada_column
result = result.sort_values(by='corr')
result = merged_table.loc[result.index,columns_to_select].dropna()
result

: 

In [ ]:
plt.figure(figsize=(5, 50))
sns.heatmap(result['corr'].to_numpy()[:,None],yticklabels=result['origin'], annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix Heatmap')
plt.show()

: 